In [242]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns
from ModelTree import ModelTree
from collections import Counter
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")

## Data exploring

In [194]:
df = pd.read_excel("stocks.xlsx", sheet_name="4th period")
names = df.iloc[0, :]
df = pd.read_excel("stocks.xlsx", sheet_name="4th period", names=names)
df.drop(axis=0, labels=[0], inplace=True)

df2 = pd.read_excel("stocks.xlsx", sheet_name="3rd period")
names = df2.iloc[0, :]
df2 = pd.read_excel("stocks.xlsx", sheet_name="3rd period", names=names)
df2.drop(axis=0, labels=[0], inplace=True)

df3 = pd.read_excel("stocks.xlsx", sheet_name="2nd period")
names = df3.iloc[0, :]
df3 = pd.read_excel("stocks.xlsx", sheet_name="2nd period", names=names)
df3.drop(axis=0, labels=[0], inplace=True)

df4 = pd.read_excel("stocks.xlsx", sheet_name="1st period")
names = df4.iloc[0, :]
df4 = pd.read_excel("stocks.xlsx", sheet_name="1st period", names=names)
df4.drop(axis=0, labels=[0], inplace=True)

Combine the dataframes for all periods in the dataset and drop ID column which will be later reset as the index

In [195]:
df = pd.concat([df, df2, df3, df4])
df.drop(axis=1, labels=["ID"], inplace=True)

Drop the columns which represent the target variables with absolute values, which leaves us working with the ones containing normalized stocks' performance indicators

In [196]:
df.drop(axis=1, columns=df.columns[[6,7,8,9,10,11]], inplace=True)
df.reset_index(drop=True, inplace=True)
df.columns = ['LargeB/P', 'LargeROE', 'LargeS/P', 'LargeReturnRateInTheLastQuarter', "LargeMarketValue", 'SmallSystematicRisk', 'AnnualReturn', 'ExcessReturn', 'SystematicRisk', 'TotalRisk', 'AbsWinRate', 'RelWinRate']

Check the columns data types and see if any changes are needed

In [197]:
df.dtypes

# all columns have "object" data type, so we need to change them to float
for col in df.columns: 
    df[col] = pd.to_numeric(df[col])

Take a look at the data

In [198]:
df.describe()

,LargeB/P,LargeROE,LargeS/P,LargeReturnRateInTheLastQuarter,LargeMarketValue,SmallSystematicRisk,AnnualReturn,ExcessReturn,SystematicRisk,TotalRisk,AbsWinRate,RelWinRate
count,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000
mean,0.166619,0.166619,0.166619,0.166619,0.166619,0.166619,0.542936,0.568207,0.427889,0.430264,0.537143,0.541964
std,0.198110,0.198110,0.198110,0.198110,0.198110,0.198110,0.145835,0.135865,0.140281,0.146439,0.140391,0.142311
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.450578,0.486764,0.325702,0.319333,0.457143,0.425000
50%,0.167000,0.167000,0.167000,0.167000,0.167000,0.167000,0.555995,0.580902,0.411967,0.399398,0.560000,0.533333
75%,0.333000,0.333000,0.333000,0.333000,0.333000,0.333000,0.655166,0.671781,0.505836,0.516704,0.628571,0.650000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.800000,0.800000,0.800000,0.800000,0.800000,0.800000


## Regression

In [199]:
df = shuffle(df)

In [200]:
y = np.array(df.pop('AnnualReturn'))
X = df.iloc[:, [0,1,2,3,4,5]]

In [174]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

In [175]:
ScikitDecisionTreeModel=DecisionTreeRegressor(max_depth=50, min_samples_split=2)
ScikitDecisionTreeModel.fit(x_train, y_train)

y_pred=ScikitDecisionTreeModel.predict(x_test)

print("Mean Squared Error: " + str(mean_squared_error(y_test, y_pred)))
print("Mean Absolute Error: " + str(mean_absolute_error(y_test, y_pred)))

Mean Squared Error: 0.03858032736233476
Mean Absolute Error: 0.1585568539852644


### Get the most important features

In [176]:
features_importance=ScikitDecisionTreeModel.feature_importances_
indexes=[]
for i in range(len(features_importance)):
    # select only the features that have any relevance
    if features_importance[i]!=0:
        indexes.append(i)
relevant_features=[]
for i in range(len(X.columns)):
    if i in indexes:
        relevant_features.append(df.columns[i])
relevant_features

['LargeB/P',
 'LargeROE',
 'LargeS/P',
 'LargeReturnRateInTheLastQuarter',
 'LargeMarketValue',
 'SmallSystematicRisk']

Since we only have so few features (6), all got selected and filtered as "important" by the decision tree.

### Cross-Validation on scikit's tree implementation

In [177]:
ScikitDecisionTreeModel=DecisionTreeRegressor(max_depth=50, min_samples_split=2)
scores_mse = cross_val_score(ScikitDecisionTreeModel, X, y, scoring='neg_mean_squared_error', cv=5)
scores_mae = cross_val_score(ScikitDecisionTreeModel, X, y, scoring='neg_mean_absolute_error', cv=5)

print("Mean Squared Error: " + str(sum(scores_mse)*(-1)/len(scores_mse)))
print("Mean Absolute Error: " + str(sum(scores_mae)*(-1)/len(scores_mae)))

Mean Squared Error: 0.02868441582928566
Mean Absolute Error: 0.13854598079457142


### Custom Tree Regression implementation

In [184]:
class CustomDecisionTreeRegressor():
    """
    Class to build a regression decision tree
    """
    def __init__(
        self, 
        Y: list,
        X: pd.DataFrame,
        min_samples_split=None,
        max_depth=None,
        depth=None,
        node_type=None,
        rule=None
    ):
        # save the data to the node 
        self.Y = Y 
        self.X = X

        # save the hyper parameters, define default values
        self.min_samples_split = min_samples_split if min_samples_split else 2
        self.max_depth = max_depth if max_depth else 10

        # default current depth of node 
        self.depth = depth if depth else 0

        # extract all the features
        self.features = list(self.X.columns)

        # type of node 
        self.node_type = node_type if node_type else 'root'

        # rule for spliting 
        self.rule = rule if rule else ""

        # get the mean of Y 
        self.ymean = np.mean(Y)

        # get the residuals 
        self.residuals = self.Y - self.ymean

        # calculate the mse of the node 
        self.mse = self.get_mse(Y, self.ymean)

        # save the number of observations in the node 
        self.n = len(Y)

        # initiate the left and right nodes as empty nodes
        self.left = None 
        self.right = None 

        # default values for splits
        self.best_feature = None 
        self.best_value = None 

    @staticmethod
    def get_mse(ytrue, yhat) -> float:
        """
        method to calculate the mean squared error 
        """
        # get the total number of samples
        n = len(ytrue)

        # get the residuals 
        r = ytrue - yhat 

        # square the residuals 
        r = r ** 2

        # sum the squared residuals
        r = np.sum(r)

        # return the average 
        return r / n

    @staticmethod
    def ma(x: np.array, window: int) -> np.array:
        """
        calculate the moving average of the given list. 
        """
        return np.convolve(x, np.ones(window), 'valid') / window

    def best_split(self) -> tuple:
        """
        given the X features and Y targets calculates the best split 
        for a decision tree
        """
        # create a dataset for spliting
        df = self.X.copy()
        df['Y'] = self.Y

        # get the GINI impurity for the base input 
        mse_base = self.mse

        # find which split yields the best GINI gain 
        #max_gain = 0

        # default best feature and split
        best_feature = None
        best_value = None

        for feature in self.features:
            # drop missing values
            Xdf = df.dropna().sort_values(feature)

            # sort the values and get the rolling average
            xmeans = self.ma(Xdf[feature].unique(), 2)

            for value in xmeans:
                # get the left and right ys 
                left_y = Xdf[Xdf[feature]<value]['Y'].values
                right_y = Xdf[Xdf[feature]>=value]['Y'].values

                # get the means 
                left_mean = np.mean(left_y)
                right_mean = np.mean(right_y)

                # get the left and right residuals 
                res_left = left_y - left_mean 
                res_right = right_y - right_mean

                # concatenate the residuals 
                r = np.concatenate((res_left, res_right), axis=None)

                # calculate the mse 
                n = len(r)
                r = r ** 2
                r = np.sum(r)
                mse_split = r / n

                # check if this is the best split so far 
                if mse_split < mse_base:
                    best_feature = feature
                    best_value = value 

                    # get the best gain to the current one 
                    mse_base = mse_split

        return (best_feature, best_value)

    def fit(self):
        """
        recursive method to create the decision tree
        """
        # make a df from the data 
        df = self.X.copy()
        df['Y'] = self.Y

        # if there is GINI to be gained, we split further 
        if (self.depth < self.max_depth) and (self.n >= self.min_samples_split):

            # get the best split 
            best_feature, best_value = self.best_split()

            if best_feature is not None:
                # save the best split to the current node 
                self.best_feature = best_feature
                self.best_value = best_value

                # gett the left and right nodes
                left_df, right_df = df[df[best_feature]<=best_value].copy(), df[df[best_feature]>best_value].copy()

                # create the left and right nodes
                left = CustomDecisionTreeRegressor(
                    left_df['Y'].values.tolist(), 
                    left_df[self.features], 
                    depth=self.depth + 1, 
                    max_depth=self.max_depth, 
                    min_samples_split=self.min_samples_split, 
                    node_type='left_node',
                    rule=f"{best_feature} <= {round(best_value, 3)}"
                    )

                self.left = left 
                self.left.fit()

                right = CustomDecisionTreeRegressor(
                    right_df['Y'].values.tolist(), 
                    right_df[self.features], 
                    depth=self.depth + 1, 
                    max_depth=self.max_depth, 
                    min_samples_split=self.min_samples_split,
                    node_type='right_node',
                    rule=f"{best_feature} > {round(best_value, 3)}"
                    )

                self.right = right
                self.right.fit()

    def print_info(self, width=4):
        """
        print information about one node
        """
        # define the number of spaces 
        const = int(self.depth * width ** 1.5)
        spaces = "-" * const
        
        if self.node_type == 'root':
            print("Root")
        else:
            print(f"|{spaces} Split rule: {self.rule}")
        print(f"{' ' * const}   | MSE of the node: {round(self.mse, 2)}")
        print(f"{' ' * const}   | Count of observations in node: {self.n}")
        print(f"{' ' * const}   | Prediction of node: {round(self.ymean, 3)}")   

    def print_tree(self):
        """
        print the entire tree
        """
        self.print_info() 
        
        if self.left is not None: 
            self.left.print_tree()
        
        if self.right is not None:
            self.right.print_tree()
    def predict(self, dataframe):
        """
        make predictions on a dataframe
        """
        predictions=[]
        for index, row in dataframe.iterrows():
            node=root
            while(node.left is not None and node.right is not None):

                # get the rules for that node
                left_rule=node.left.rule.split(" ")
                right_rule=node.right.rule.split(" ")
            
                # continue on the left node
                if (left_rule[1]=="<="):
                    if (row[left_rule[0]]<=float(left_rule[2])):
                        node=node.left
                elif (left_rule[1]=="<"):
                    if (row[left_rule[0]]<float(left_rule[2])):
                        node=node.left
                elif (left_rule[1]==">"):
                    if (row[left_rule[0]]>float(left_rule[2])):
                        node=node.left
                else:
                    if (row[left_rule[0]]>=float(left_rule[2])):
                        node=node.left

                # contnue on the right node       
                if (right_rule[1]=="<="):
                    if (row[right_rule[0]]<=float(right_rule[2])):
                        node=node.right
                elif (right_rule[1]=="<"):
                    if (row[right_rule[0]]<float(right_rule[2])):
                        node=node.right
                elif (right_rule[1]==">"):
                    if (row[right_rule[0]]>float(right_rule[2])):
                        node=node.right
                else:
                    if (row[right_rule[0]]>=float(right_rule[2])):
                        node=node.right

                if (node.left is None and node.right is None):
                    predictions.append(node.ymean)

        return predictions

### Holdout with CustomTreeRegressor

In [243]:
# divide the dataset into 80/20 (training/test)
threshold=int(0.8*X.shape[0])

X_train = X[:threshold]
Y_train = y[:threshold]
X_test = X[threshold:]
Y_test = y[threshold:]

In [244]:
# define the tree
root = CustomDecisionTreeRegressor(Y_train, X_train, max_depth=50, min_samples_split=2)
# fit the tree
root.fit()

In [245]:
predictions=root.predict(X_test)

In [246]:
# model evaluation using Root Mean Squared Error and Mean Absolute Error
print("Mean Squared Error: " + str(mean_squared_error(Y_test, predictions)))
print("Mean Absolute Error: " + str(mean_absolute_error(Y_test, predictions)))

Mean Squared Error: 0.024872547067057868
Mean Absolute Error: 0.13359660508763005


### Crossvalidation with CustomTreeRegressor

In [248]:
kf = KFold(n_splits=5, shuffle=False)
#define lists to store te results for each fold
maes=[]
mses=[]

i = 1
for train_index, test_index in kf.split(df):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
        
    # define the tree
    root = CustomDecisionTreeRegressor(y_train, X_train, max_depth=50, min_samples_split=2)
    # fit the tree
    root.fit()
    mse=mean_squared_error(y_test, root.predict(X_test))
    mses.append(mse)
    mae=mean_absolute_error(y_test, root.predict(X_test))
    maes.append(mae)
    
    i += 1
    
print("Average MSE: " + str(np.mean(mses)))
print("Average MAE: " + str(np.mean(maes)))

Average MSE: 0.02643021057180304
Average MAE: 0.13244258702713638


### Holdout with ModelTree

In [189]:
class linear_regr:

    def __init__(self):
        from sklearn.linear_model import LinearRegression
        self.model = LinearRegression()

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)

    def loss(self, X, y, y_pred):
        return mean_squared_error(y, y_pred)

In [209]:
# divide the dataset into 80/20 (training/test)
threshold=int(0.8*X.shape[0])

X_train = np.array(X[:threshold])
Y_train = np.array(y[:threshold])
X_test = np.array(X[threshold:])
Y_test = np.array(y[threshold:])

In [213]:
# define the tree
model = linear_regr()
model_tree = ModelTree(model, max_depth=50, min_samples_leaf=2)
model_tree.fit(X_train, Y_train)

In [217]:
Y_pred = model_tree.predict(X_test)

In [218]:
# model evaluation using Root Mean Squared Error and Mean Absolute Error
print("Mean Squared Error: " + str(mean_squared_error(Y_test, Y_pred)))
print("Mean Absolute Error: " + str(mean_absolute_error(Y_test, Y_pred)))

Mean Squared Error: 0.024872547067057774
Mean Absolute Error: 0.1335966050876297


### Crossvalidation with ModelTree

In [249]:
kf = KFold(n_splits=5, shuffle=False)
#define lists to store te results for each fold
maes=[]
mses=[]

i = 1
for train_index, test_index in kf.split(df):
    X_train = np.array(X.iloc[train_index])
    X_test = np.array(X.iloc[test_index])
    Y_train = np.array(y[train_index])
    Y_test = np.array(y[test_index])
        
    model = linear_regr()
    model_tree = ModelTree(model, max_depth=50, min_samples_leaf=2)
    model_tree.fit(X_train, Y_train)
    
    mse=mean_squared_error(Y_test, model_tree.predict(X_test))
    mses.append(mse)
    mae=mean_absolute_error(Y_test, model_tree.predict(X_test))
    maes.append(mae)
    
    i += 1

print("Average MSE: " + str(np.mean(mses)))
print("Average MAE: " + str(np.mean(maes)))

Average MSE: 0.026429292812349865
Average MAE: 0.13243855189428613


### Holdout with Linear Regression

In [235]:
model = LinearRegression()

# divide the dataset into 80/20 (training/test)
threshold=int(0.8*X.shape[0])

X_train = np.array(X[:threshold])
Y_train = np.array(y[:threshold])
X_test = np.array(X[threshold:])
Y_test = np.array(y[threshold:])

model.fit(X_train, Y_train)

Y_pred = model_tree.predict(X_test)

# model evaluation using Root Mean Squared Error and Mean Absolute Error
print("Mean Squared Error: " + str(mean_squared_error(Y_test, Y_pred)))
print("Mean Absolute Error: " + str(mean_absolute_error(Y_test, Y_pred)))

Mean Squared Error: 0.02476966150620715
Mean Absolute Error: 0.1330597672645201


### Crossvalidation with Linear Regression

In [250]:
kf = KFold(n_splits=5, shuffle=False)
#define lists to store te results for each fold
maes=[]
mses=[]

i = 1
for train_index, test_index in kf.split(df):
    X_train = np.array(X.iloc[train_index])
    X_test = np.array(X.iloc[test_index])
    Y_train = np.array(y[train_index])
    Y_test = np.array(y[test_index])
        
    model = LinearRegression()
    model.fit(X_train, Y_train)
    
    mse=mean_squared_error(Y_test, model_tree.predict(X_test))
    mses.append(mse)
    mae=mean_absolute_error(Y_test, model_tree.predict(X_test))
    maes.append(mae)
    
    i += 1

print("Average MSE: " + str(np.mean(mses)))
print("Average MAE: " + str(np.mean(maes)))

Average MSE: 0.016181991548173322
Average MAE: 0.10362028774197374


### Holdout with Random Forest Regressor 

In [251]:
model = RandomForestRegressor(max_depth=50, min_samples_leaf=2)

# divide the dataset into 80/20 (training/test)
threshold=int(0.8*X.shape[0])

X_train = np.array(X[:threshold])
Y_train = np.array(y[:threshold])
X_test = np.array(X[threshold:])
Y_test = np.array(y[threshold:])

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

# model evaluation using Root Mean Squared Error and Mean Absolute Error
print("Mean Squared Error: " + str(mean_squared_error(Y_test, Y_pred)))
print("Mean Absolute Error: " + str(mean_absolute_error(Y_test, Y_pred)))

Mean Squared Error: 0.023621476207673436
Mean Absolute Error: 0.1312919287807662


### Crossvalidation with Random Forest Regressor

In [252]:
kf = KFold(n_splits=5, shuffle=False)
maes=[]
mses=[]

i = 1
for train_index, test_index in kf.split(df):
    X_train = np.array(X.iloc[train_index])
    X_test = np.array(X.iloc[test_index])
    Y_train = np.array(y[train_index])
    Y_test = np.array(y[test_index])
        
    model = RandomForestRegressor(max_depth=50, min_samples_leaf=2)
    model.fit(X_train, Y_train)
    
    mse=mean_squared_error(Y_test, model_tree.predict(X_test))
    mses.append(mse)
    mae=mean_absolute_error(Y_test, model_tree.predict(X_test))
    maes.append(mae)

    i += 1

print("Average MSE: " + str(np.mean(mses)))
print("Average MAE: " + str(np.mean(maes)))

Average MSE: 0.016181991548173322
Average MAE: 0.10362028774197374
